# All Imports

In [3]:
import os
from dotenv import load_dotenv
import requests
import time
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Semaphore
import threading

# Globals

In [4]:
DEFAULT_WR = 0.40
MAX_REQUESTS_PER_SECOND = 20
MAX_REQUESTS_PER_2MINUTE = 100
# Can use later for getting better avg/accounting for other gamemodes
EXCLUDED_QUEUE_IDS = {
    0,    # Custom games
    830,  # Co-op vs. AI: Intro bots
    840,  # Co-op vs. AI: Beginner bots
    850,  # Co-op vs. AI: Intermediate bots
    450,  # ARAM
    900,  # ARURF
    920,  # Nexus Blitz
    1300  # Nexus Blitz (old)
    # Add more queue IDs to exclude other game modes
}

# Getting Data

In [5]:
# Set up API (NA1 Region Only)
load_dotenv('.env')
api_key = os.getenv("RIOT_API_KEY")

In [6]:
# Testing key
#print(api_key)

In [7]:
# Custom RateLimiter class
class RateLimiter:
    def __init__(self, max_calls_1, period_1, max_calls_2, period_2):
        # Init for keeping tracking of calls per second (period_1) and 2min (period2)
        self.max_calls_1 = max_calls_1
        self.period_1 = period_1
        self.calls_1 = 0
        self.start_time_1 = time.time()

        self.max_calls_2 = max_calls_2
        self.period_2 = period_2
        self.calls_2 = 0
        self.start_time_2 = time.time()

    def acquire(self):
        # Called everytime right before API is used to time it and ensure no requests more than x per sec and y per 2min
        current_time = time.time()

        # Find time elapsed since first 
        elapsed_1 = current_time - self.start_time_1
        elapsed_2 = current_time - self.start_time_2

        # Reset calls and start time if period passed
        if elapsed_1 > self.period_1:
            self.calls_1 = 0
            self.start_time = current_time

        if elapsed_2 > self.period_2:
            self.calls_2 = 0
            self.start_time_2 = current_time

        # Proceed to sleep or not depending on if max calls per second exceeded
        if self.calls_1 < self.max_calls_1:
            self.calls_1 += 1
        else:
            time_to_wait = self.period_1 - elapsed_1
            if time_to_wait > 0:
                print(f"Rate limit reached for {MAX_REQUESTS_PER_SECOND}/1s. Sleeping for {time_to_wait:.2f} seconds.")
                time.sleep(time_to_wait)
            self.calls_1 = 1
            self.start_time_1 = time.time()

        # Proceed to sleep or not depending on if max calls per 2min exceeded
        if self.calls_2 < self.max_calls_2:
            self.calls_2 += 1
        else:
            time_to_wait = self.period_2 - elapsed_2
            if time_to_wait > 0:
                print(f"Rate limit reached for {MAX_REQUESTS_PER_2MINUTE}/2m. Sleeping for {time_to_wait:.2f} seconds.")
                time.sleep(time_to_wait)
            self.calls_2 = 1
            self.start_time_2 = time.time()

In [13]:
# Function for detecting and handling limit reached (429) and NOT success (200)
def apiCallHandler(request_url, rate_limiter, stop_event):
    #print(request_url)

    rate_limiter.acquire()
    
    #if stop_event.is_set():
        #print("Stopping because event flag")
        #sys.exit("stopping")

    headers = {
        "X-Riot-Token": api_key 
    }
    
    response = requests.get(request_url, headers=headers)

    numFailedRetries = 0
    while response.status_code != 200: # while loop here for later when we want to ignore error 429
        # Retry limiter
        if(numFailedRetries >= 2):
            sys.exit(f"Exceeded retry limit of {numRetries}")
            
        if(response.status_code == 429):
            # Not success but is 429 API limit error
            print("TMP: status 429 detected")
            retry_after = int(response.headers.get("Retry-After", 0))
            #stop_event.set()
        else:
            # Not success and not 429 API limit error
            print(f"Failed to fetch data: {response.status_code}")
            sys.exit("Stopping all execution")

        # 429, retry
        print(f"Retrying in {retry_after}")
        time.sleep(retry_after)
        numFailedRetries += 1
        rate_limiter.acquire()
        response = requests.get(request_url, headers)
        
    # (finally) status of 200
    return response.json()

In [9]:
# Function for using multithreading when calling multiple APIs
# urls = a list of urls desired (and compatible) to multithread
def multithread_call(urls, rate_limiter, stop_event):
    results = []
    # Threadpoolexecutor ensures max concurrent workers don't exceed
    with ThreadPoolExecutor(max_workers=MAX_REQUESTS_PER_SECOND) as executor:
        # "Future" objects store the future value of the API call
        futures = [executor.submit(apiCallHandler, f"https://americas.api.riotgames.com/lol/match/v5/matches/{url}?api_key={api_key}", rate_limiter, stop_event) for url in urls]
        # as_completed takes "Future" objects in the order they complete
        for future in as_completed(futures):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                # This shouldn't ever happen, API error checking done in the API handler
                print(f"Exception: {e}")
                sys.exit("Stopping all execution")
    return results

In [10]:
# Function for calculating average win rate of a summoner using multithreading
def avg_wr_summoner(match_history, summoner_puuid, rate_limiter, stop_event):
    if(len(match_history) == 0):
        return DEFAULT_WR
    results = multithread_call(match_history, rate_limiter, stop_event)
    total_matches = 0
    win_count = 0
    for response in results:
        try:
            # Finds the first participant's id who's equal to the summoner's puuid
            participant = next(p for p in response['info']['participants'] if p['puuid'] == summoner_puuid)
            if(participant['win'] == True):
                win_count+=1
            total_matches+=1
        except StopIteration:
            print("ERROR: StopIteration exception occured, Riot data incorrect?")
            sys.exit("Stopping all execution")
        except:
            print("ERROR: Exception occured, shouldn't be here")
            sys.exit("Stopping all execution")
    return round(win_count/total_matches, 2)

In [11]:
# Gets Data for a given summoner PUUID, and their champ played (for champ mastery)
# Data currently equals: Summoner Level, match history length (max 20), AVG win rate for past 20 matches, champ mastery
def get_summoner_features(summoner_puuid, champion_played, rate_limiter, stop_event):
    
    # Get summoners' match history & length using their PUUID, filters only for normal draft games; max api_calls = 1 (No multithread)
    summoner_match_history = apiCallHandler(f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{summoner_puuid}/ids?queue=400&type=normal&start=0&count=20&api_key={api_key}', rate_limiter, stop_event)
    match_history_length = len(summoner_match_history)

    # All calls below can be multithreaded since prev info is not needed
    # urls = [] Future work: have all possible multi threaded calls do one call
    
    # Get summoners' level using their PUUID; max api_calls = 1
    summoner_data = apiCallHandler(f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{summoner_puuid}?api_key={api_key}', rate_limiter, stop_event)
    level = summoner_data['summonerLevel']
    
    # Get summoners' win rate using match history; max_api_calls = 20 
    # Future work: Ensure that no custom, bot, tutorial, or arena/limited game mode matches
    # Future work: If match count is < 20, THEN use quickplay and aram stats (assign weights to them)
    # Future work: add compatibility for predicting specifically gamemodes other than normal draft 
    avg_wr = avg_wr_summoner(summoner_match_history, summoner_puuid, rate_limiter, stop_event)

    # Get summoners' champ mastery for the match 
    champ_mastery = 0
    return (level, match_history_length, avg_wr, champ_mastery)

In [14]:
# Gets the features given the match ID
#def get_features(match_id):
# Rate_limiters for limiting requests
# sleep for 2min here in future just in case, or pass rate_limiter into this function too (probs this)
rate_limiter = RateLimiter(max_calls_1=MAX_REQUESTS_PER_SECOND, period_1=1, max_calls_2=MAX_REQUESTS_PER_2MINUTE, period_2=120)


# Event for signaling all threads to stop
stop_event = threading.Event()

match_id = 'NA1_5024880870' #replace later
match_info = apiCallHandler(f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}', rate_limiter, stop_event)

players = match_info['metadata']['participants']

# todo: get avg rank too
# add flag status for api multithread failure
# fix current error

for player in players:
    level, match_history_length, avg_wr, champ_mastery = get_summoner_features(player, 0, rate_limiter, stop_event)
    print(f'Summoner lvl: {level}')
    print(f'Summoner match history len: {match_history_length}')
    print(f'Summoner avg wr: {avg_wr}')
    print(f'Summoner champ mastery: {champ_mastery}')


Summoner lvl: 158
Summoner match history len: 20
Summoner avg wr: 0.35
Summoner champ mastery: 0
Summoner lvl: 290
Summoner match history len: 20
Summoner avg wr: 0.35
Summoner champ mastery: 0
Summoner lvl: 290
Summoner match history len: 20
Summoner avg wr: 0.5
Summoner champ mastery: 0
Summoner lvl: 482
Summoner match history len: 20
Summoner avg wr: 0.5
Summoner champ mastery: 0
Rate limit reached for 100/2m. Sleeping for 95.59 seconds.
Rate limit reached for 100/2m. Sleeping for 95.59 seconds.
Rate limit reached for 100/2m. Sleeping for 95.59 seconds.
Rate limit reached for 100/2m. Sleeping for 95.58 seconds.
Rate limit reached for 100/2m. Sleeping for 95.58 seconds.
Rate limit reached for 100/2m. Sleeping for 95.58 seconds.
Rate limit reached for 100/2m. Sleeping for 95.58 seconds.
Rate limit reached for 100/2m. Sleeping for 95.58 seconds.
Rate limit reached for 100/2m. Sleeping for 95.58 seconds.
Rate limit reached for 100/2m. Sleeping for 95.58 seconds.
Rate limit reached for 1

# Placing Data Into CSV File

# Loading Data